# T4 Clustering: algorisme K-mitjanes

# Índex

1. Clustering particional
2. Criteri suma d'errors quadràtics
3. Algorisme $K$-mitjanes de Duda i Hart
4. Algorisme $K$-mitjanes convencional

# 1 Clustering particional

**Clustering particional:** $\;$ donat un conjunt de $N$ dades $\mathcal{D}$ i un nombre de clústers $K,\,$ el clustering particional consisteix a optimitzar alguna funció criteri $J(\Pi)$ per a avaluar la qualitat de qualsevol partició $\Pi$ de les dades en $K$ clústers
$$\Pi^* = \operatorname*{argopt}_{\Pi} J(\Pi)$$

**Intractabilitat:** $\;$ el clustering particional és en general un problema intractable ja que el nombre de particions a explorar creix exponencialment amb $N$ i $K$ (veure [nombres d'Stirling del segon tipus](https://en.wikipedia.org/wiki/Stirling_numbers_of_the_second_kind))

**Aproximació usual:** $\;$ fem ús d'algorismes aproximats per a optimitzar un criteri particular com ara la suma d'errors quadràtics

# 2 Criteri suma d'errors quadràtics

**Suma d'errors quadràtics (SEQ):** $\;$ d'una partició $\Pi=\{X_1,\dotsc,X_K\}$
$$J(\Pi) = \sum_{k=1}^K J_k%
\quad\text{amb}\quad%
J_k = \sum_{\boldsymbol{x}\in X_k} \lVert{\boldsymbol{x}-\boldsymbol{m}_k}\rVert_2^2
\quad\text{i}\quad%
\boldsymbol{m}_k = \frac{1}{\lvert X_k\rvert}\sum_{\boldsymbol{x}\in X_k} \boldsymbol{x}$$

**Interpretació:**
* Cada clúster $k$ es representa pel seu **centroide** o **mitjana** $\,\boldsymbol{m}_k$
* Si $\boldsymbol{x}$ pertany al clúster $k$, $\boldsymbol{x}-\boldsymbol{m}_k$ és el **vector error** obtés al representar $\boldsymbol{x}$ amb $\boldsymbol{m}_k$
* L'error associat a $\boldsymbol{x}$ es mesura amb la norma Euclidiana del seu vector error, $\lVert{\boldsymbol{x}-\boldsymbol{m}_k}\rVert_2$
* Anomenem **distorsió** del clúster $k$, $\,J_k,\,$ a la suma d'errors al quadrat de les seues dades
* El criteri SEQ és la suma de les distorsions de tots els clústers i, òbviament, és un criteri a minimitzar
* Idealment, esperem clústers hiper-esfèrics compactes i de grandària pareguda, al voltant de $K$ mitjanes ben separades
* Si la partició natural de les dades és distinta a l'esperada, és probable que la minimització de la SEQ no la trobe

**Exemple:** $\;$ càlcul de la SEQ per a $\Pi=\{X_1=\{(1, 7)^t, (4, 2)^t, (4,6)^t\}, X_2=\{(8, 2)^t, (8, 6)^t\}\}$
$$\begin{align*}
\boldsymbol{m}_1&=(3,5)^t&&J_1=8+10+2=20\\
\boldsymbol{m}_2&=(8,4)^t&&J_2=4+4=8
\end{align*}$$
$$J=J_1+J_2=28$$

In [1]:
import numpy as np
X = np.array([[1, 7], [4, 2], [4, 6], [8, 2], [8, 6]]); y = np.array([1, 1, 1, 2, 2]); J = 0.0;
for k in np.unique(y):
    Xk = X[np.where(y==k),:].reshape(-1, X.shape[1])
    mk = Xk.mean(axis=0)
    Jk = np.square(Xk - mk).sum()
    print(f'Clúster {k}: m = {mk} J = {Jk}')
    J += Jk
print(f'J = {J}')

Clúster 1: m = [3. 5.] J = 20.0
Clúster 2: m = [8. 4.] J = 8.0
J = 28.0


# 3 Algorisme $K$-mitjanes de Duda i Hart

**Increment de SEQ al transferir una dada de clúster:** $\;$ si es transfereix una dada $\boldsymbol{x}$ del clúster $i$ al $j$, l'increment de SEQ és
$$\Delta J = \frac{\lvert X_j\rvert}{\lvert X_j\rvert + 1}\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2-\frac{\lvert X_i\rvert}{\lvert X_i\rvert - 1}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$$

**Transferència profitosa:** $\;$ la transferència de $\boldsymbol{x}$ del clúster $i$ al $j$ serà profitosa si $\Delta J<0,$ és a dir, si
$$\frac{\lvert X_j\rvert}{\lvert X_j\rvert + 1}\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2<\frac{\lvert X_i\rvert}{\lvert X_i\rvert - 1}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$$

**Algorisme $K$-mitjanes de Duda i Hart:** $\;$ cerca transferències profitoses i les aplica fins que no en queden
> **Entrada:** $\;$ una partició inicial, $\,\Pi=\{X_1,\dotsc,X_K\}$ <br>
> **Eixida:** $\;$ una partició optimitzada, $\,\Pi^*=\{X_1,\dotsc,X_K\}$ <br>
> Calcular mitjanes i $J$ <br>
> `repetir`
>> `per a tota` $\;$ dada $\,\boldsymbol{x}$ <br>
>>> Siga $i$ el clúster en el qual es troba $\boldsymbol{x}$ <br>
>>> Trobar un $j^*\neq i$ que minimitze $\,\triangle J\,$ en transferir $\,\boldsymbol{x}\,$ d'$i$ a $j^*$ <br>
>>> `si` $\,\triangle J<0:\;$ transferir $\boldsymbol{x}$ d'$i$ a $j^*$ i actualitzar mitjanes i $J$ <br>
>
> `fins que` $\;$ no es trobe cap transferència profitosa

**Exemple (cont.):** $\quad X_1=\{\boldsymbol{x}_1=(1, 7)^t, \boldsymbol{x}_2=(4, 2)^t, \boldsymbol{x}_3=(4,6)^t\}\quad X_2=\{\boldsymbol{x}_4=(8, 2)^t, \boldsymbol{x}_5=(8, 6)^t\}$
<center>

|Iteració|$\boldsymbol{x}$|$i$|$j^*$|$\frac{\lvert X_i\rvert}{\lvert X_i\rvert - 1}\lVert\boldsymbol{x}-\boldsymbol{m}_i\rVert_2^2$|$\frac{\lvert X_j\rvert}{\lvert X_j\rvert + 1}\lVert\boldsymbol{x}-\boldsymbol{m}_j\rVert_2^2$|$\triangle J$|$X_1$|$X_2$|$\boldsymbol{m}_1$|$\boldsymbol{m}_2$|$J$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
||||||||$\{\boldsymbol{x}_1,\boldsymbol{x}_2,\boldsymbol{x}_3\}$|$\{\boldsymbol{x}_4,\boldsymbol{x}_5\}$|$(3,5)^t$|$(8,4)^t$|$28$|
|$1$|$\boldsymbol{x}_1$|$1$|$2$|$\frac{3}{2}\cdot 8$|$\frac{2}{3}\cdot 58$|$\frac{80}{3}$||||
|$1$|$\boldsymbol{x}_2$|$1$|$2$|$\frac{3}{2}\cdot 10$|$\frac{2}{3}\cdot 20$|$-\frac{5}{3}$||||
</center>

# 4 Algorisme $K$-mitjanes convencional